# Numerical Computation Project

## Sudoku Solver (Linear Programming)

### Rathanon Singh Gandhi(6280205)

In [ ]:
#pip install pulp

In [ ]:
import pulp as plp
from colorama import Fore

In this project we are going to use linear optimization or linear programming to solve an unsolved sudoku. We will be using PuLP package as our linear programming package.
The rules for Sudoku are simple, we just put in numbers in the grid just and make sure no numbers repeat in the same grid, row or columns.
Linear Programming consists of the following
1. An objective function
2. Descision Variables
3. Constrains



The goal of linear programming is to find an optimal set of decision variables for the given objective function with all the constraints applied. The solution could be a feasible solution or an optimal solution.



We shall compare these concepts of linear programming with our sudoku solver.
<br>
1. An objective function - Normally in linear programming, we have an objective function that we try to either maximize or minimize. But in this case, we do not have any objective function. It is more of a feasibility problem, i.e. if the constraints are satisfied then the sudoku is solved. So we will have to create a fake or dummy objective.
<br>
2. Descision Variables - Sudoku grid consists of 81 cells (9x9 grid). Each cell can take a value between 1 and 9. If we create a set of boolean decision variables for each of these values, then we get a total of 729 variables (9x9x9). In the case of our solver we have named them r,c,g for rows, colums and grid respectively.
<br> 
3. Constrains-The rules for Sudoku need to be set as constraints to solve this problem.Given that Sudoku is a 9x9 grid, the rules of the game are mentioned below:
<br>
    Constraint 1: Each cell should be filled with a single value between 1 and 9

    Constraint 2: Each row should contain every number from 1 to 9 once

    Constraint 3: Each column should contain every number from 1 to 9 once

    Constraint 4: Each 3x3 grid, starting from top left, should contain every number from 1 to 9 once

This below is our Main input function where we take in the input sudoku grid .

We initilise our problem with regards to linear programming. 

We create our descision variable of rows colums and grid which will ranre from 0 to 8 as there are 9 rows,9 colums and 9 in a grid. We also have vairable n which ranges from 1 to 9 as the numbers we can place in the blank cell. 

We also create a Dummy objective here due to the fact that no optimisation is need in sudoku. We just need a feasible solution.

In [ ]:
def solve_sudoku(input_sudoku):
    # Create the linear programming problem for us to solve it
    problem = plp.LpProblem("Sudoku_Solver")

    r = range(0,9) #rows variable
    c = range(0,9) #columns variable
    g = range(0,9) #grid variable
    n = range(1,10) #soduku values(numbers) from 1 to 9

    # Decision Variable
    gridVariable = plp.LpVariable.dicts("grid_value", (r,c,n), cat='Binary')


    # There is no objective function that we are trying maximize or minimize so we simply S\set a dummy objective as sudoku only works on it rules i.e. constraints
    obj = plp.lpSum(0) #objective
    problem.setObjective(obj)

    # a default function which adds the rules of the game as contraints for our linear program
    add_default_sudoku_constraints(problem, gridVariable, r, c, g, n)

    # a fuctions that makes the already filled cells as additional contraints for our linear brogram
    add_prefilled_constraints(problem, input_sudoku, gridVariable, r, c, n)

    # Solve the problem using tlinear programming
    problem.solve()

    # Find the status of the sudoku which determine wheter the input sudoku is solvable or not
    status = plp.LpStatus[problem.status]
    
    # Extract and print the final solution if the status is optimal else print appropriate message
    if status == 'Optimal':
        print(f'Solution Status = {plp.LpStatus[problem.status]}')
        sol = extract_solution(gridVariable, r, c, n)
        print_solution(sol, r,c,input_sudoku)
    else:
        print("The input sudoku grid was invalid. Please put a valid grid")
        print(f'Solution Status = {plp.LpStatus[problem.status]}')



This below is our constrains funtion. As we know the constrains are the rules of the game. We go through each rules and add them as contrains for our linear programming solver. There are 4 constrains in total as already metioned above.

In [ ]:

def add_default_sudoku_constraints(prob, grid_vars, rows, cols, grids, values):
    # Constraint to ensure only one value can be filled in one cell
    lr = len(rows)
    for r in range(0,lr):
        lc = len(cols)
        for c in range(0,lc):
            prob.addConstraint(plp.LpConstraint(e=plp.lpSum([grid_vars[r][c][value] for value in values]),
                                                sense=plp.LpConstraintEQ, rhs=1, name=f"constraint_sum_{r}_{c}"))

    # Constraint to ensure that values from 1 to 9 is filled only once in a row so that no repetitions in same row
    for r in range(0,lr):
        for v in values:
            prob.addConstraint(plp.LpConstraint(e=plp.lpSum([grid_vars[r][col][v] * v for col in cols]),
                                                sense=plp.LpConstraintEQ, rhs=v,
                                                name=f"constraint_uniq_row_{r}_{v}"))

    # Constraint to ensure that values from 1 to 9 is filled only once in a column so that no repetitions in same column

    for c in range(0,lc):
        for v in values:
            prob.addConstraint(plp.LpConstraint(e=plp.lpSum([grid_vars[row][c][v] * v for row in rows]),
                                                sense=plp.LpConstraintEQ, rhs=v,
                                                name=f"constraint_uniq_col_{c}_{v}"))

    # Constraint to ensure that values from 1 to 9 is filled only once in the 3x3 grid so that no repetitions in same sub grid.
    for grid in grids:
        gr = int(grid / 3) #grid rows
        gc = int(grid % 3) #grid columns

        for v in values:
            prob.addConstraint(plp.LpConstraint(e=plp.lpSum(
                [grid_vars[gr * 3 + row][gc * 3 + col][v] * v for col in range(0, 3) for row in
                 range(0, 3)]),sense=plp.LpConstraintEQ, rhs=v, name=f"constraint_uniq_grid_{grid}_{v}"))


Now this function deals with one additional contraints. This contrains are the numbers that already filled in in the sudoku puzzle. We cannot change those values and we fill in the blank cell with the above contrains with regards to the values that are already prefilled in the puzzle. So we add that number that is not zero in as additional constraint. Whem we will solve the zeros are treated as blank cells and will be replaced with the feasible number.

In [ ]:
def add_prefilled_constraints(prob, input_sudoku, grid_vars, rows, cols, values):
    lr = len(rows)
    for r in range(0,lr):
        lc = len(cols)
        for c in range(0,lc):
            if(input_sudoku[r][c] != 0):#if the input sudoku cell value is not zero we add that cell as an additional constraint
                prob.addConstraint(plp.LpConstraint(e=plp.lpSum([grid_vars[r][c][value]*value  for value in values]),
                                                    sense=plp.LpConstraintEQ,
                                                    rhs=input_sudoku[r][c],
                                                    name=f"constraint_prefilled_{r}_{c}"))

This function helps us extract the solution after we solved our problem and store it in a 2d list called sol. So we simply loop through each cell and then we loop through the values from 1 to 9 and if that value is feasible after we sovled our proble, we put the value in the row and column index. So by the end of our function we will have the final solved sudoku grid stored in sol.

In [ ]:
def extract_solution(grid_vars, rows, cols, values):
    sol = [[0 for col in cols] for row in rows] #2d list to store our solution
    lr = len(rows)
    for r in range(0,lr):
        lc = len(cols)
        for c in range(0,lc):
            for value in values:
                if plp.value(grid_vars[r][c][value]):
                    sol[r][c] = value #filling the blank cells with feasible values
    return sol

In [ ]:
def print_solution(solution, rows,cols,input_sudoku):
    # Print the final result in form of grids made using symbols
    print(f"\nFinal Sodoku Solution: \n\033[1;35mPrefilled numbers denoted with Purple. \033[1;32m\nSolution is denoted with Green")
    i=0
    for row in rows:
        print("\033[1;34m\n",end="\n|  ")
        for col in cols:
            num_end = "  |  " if ((col+1)%3 == 0) else "   "
            if(solution[row][col]==input_sudoku[row][col]):
                print("\033[1;35m",solution[row][col],end="\033[1;34m"+num_end)
            else:
                print("\033[1;32m",solution[row][col],end="\033[1;34m"+num_end)
        i=i+1
        if(i==3 or i==6):
            print("\033[1;34m\n\n+ -------------- + -------------- + -------------- +",end="")      

In [14]:
normal_sudoku = [
                    [3,0,0,8,0,0,0,0,1],
                    [0,0,0,0,0,2,0,0,0],
                    [0,4,1,5,0,0,8,3,0],
                    [0,2,0,0,0,1,0,0,0],
                    [8,5,0,4,0,3,0,1,7],
                    [0,0,0,7,0,0,0,2,0],
                    [0,8,5,0,0,9,7,4,0],
                    [0,0,0,1,0,0,0,0,0],
                    [9,0,0,0,0,7,0,0,6]
                
                ]


solve_sudoku(input_sudoku=normal_sudoku)

The input sudoku grid was invalid. Please put a valid grid
Solution Status = Infeasible
